In [1]:
import sys
import os
from datetime import datetime, timezone
from uuid import uuid4
import yaml
import json
from typing import List, Dict, Any
from pathlib import Path

os.chdir('..') 

from src.core.config.agent_config_manager import AgentConfigManager
from src.core.context_manager import SQLiteContextProvider
from src.nodes.answer_agent import AnswerAgentNode
from src.interfaces.core.state_schema import HybridSystemState

In [2]:
class InteractiveAgentTester:
    """Interactive testing interface for the agent system"""
    
    def __init__(self, config_dir: str = "config"):
        self.config_dir = Path(config_dir)
        self.config_manager = None
        self.context_manager = None
        self.answer_agent = None
        self.current_settings = {}
        
    def initialize(self):
        """Initialize the agent system"""
        print("🚀 Initializing Agent Testing System...")
        
        # Initialize configuration manager
        self.config_manager = AgentConfigManager(str(self.config_dir))
        
        # Initialize context manager
        self.context_manager = SQLiteContextProvider("interactive_test.db")
        
        # Create answer agent
        self.answer_agent = AnswerAgentNode(self.config_manager, self.context_manager)
        
        # Load current settings
        self._load_current_settings()
        
        print("✅ System initialized successfully!")
        
    def _load_current_settings(self):
        """Load current configuration settings"""
        agent_config_path = self.config_dir / "agents" / "answer_agent" / "config.yaml"
        models_config_path = self.config_dir / "shared" / "models.yaml"
        
        with open(agent_config_path, 'r') as f:
            agent_config = yaml.safe_load(f)
            
        with open(models_config_path, 'r') as f:
            models_config = yaml.safe_load(f)
            
        self.current_settings = {
            'agent_config': agent_config,
            'models_config': models_config
        }
        
    def display_current_settings(self):
        """Display current agent configuration"""
        agent_config = self.answer_agent.agent_config
        
        print("\n" + "="*80)
        print("🔧 CURRENT AGENT CONFIGURATION")
        print("="*80)
        
        print(f"Agent: {agent_config.name}")
        print(f"Description: {agent_config.description}")
        print(f"Type: {agent_config.type}")
        
        print(f"\n📊 MODEL SETTINGS:")
        print(f"  Preferred Model: {agent_config.get_preferred_model()}")
        fallback_models = agent_config.get_fallback_models()
        print(f"  Fallback Models: {', '.join(fallback_models) if fallback_models else 'None'}")
        
        print(f"\n⚙️  GENERATION SETTINGS:")
        print(f"  Temperature: {agent_config.settings.get('temperature', 'Not set')}")
        print(f"  Max Tokens: {agent_config.settings.get('max_tokens', 'Not set')}")
        print(f"  Timeout: {agent_config.settings.get('timeout', 'Not set')}s")
        print(f"  Max Retries: {agent_config.settings.get('max_retries', 'Not set')}")
        
        print(f"\n🎯 BEHAVIOR SETTINGS:")
        print(f"  Context Integration: {agent_config.behavior.get('context_integration', False)}")
        print(f"  Response Style: {agent_config.behavior.get('response_style', 'Not set')}")
        print(f"  Personalization: {agent_config.behavior.get('personalization', False)}")
        
        print(f"\n🚨 ESCALATION SETTINGS:")
        print(f"  Confidence Threshold: {agent_config.escalation.get('confidence_threshold', 'Not set')}")
        print(f"  Auto-escalation: {agent_config.escalation.get('enable_auto_escalation', False)}")
        
        print("="*80)
        
    def get_available_models(self) -> List[str]:
        """Get list of available models"""
        return list(self.current_settings['models_config']['models'].keys())
        
    def create_test_state(self, query: str) -> HybridSystemState:
        """Create a test state for the demo"""
        return {
            "query_id": str(uuid4()),
            "user_id": "interactive_test_user",
            "session_id": f"test_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            "query": query,
            "timestamp": datetime.now(timezone.utc),
            "messages": [],
            "context": {},
            "metadata": {"interactive_test": True},
            "next_action": "answer"
        }
        
    def test_questions(self, questions: List[str], show_details: bool = True):
        """Test a list of questions with current settings"""
        print(f"\n🧪 TESTING {len(questions)} QUESTIONS")
        print("="*80)
        
        results = []
        
        for i, question in enumerate(questions, 1):
            print(f"\n--- Question {i}/{len(questions)} ---")
            print(f"❓ {question}")
            
            # Create test state
            state = self.create_test_state(question)
            
            # Get response from agent
            print("⏳ Processing...")
            start_time = datetime.now()
            
            try:
                response_state = self.answer_agent(state)
                end_time = datetime.now()
                response_time = (end_time - start_time).total_seconds()
                
                # Extract response
                ai_response = response_state.get("ai_response", "No response generated")
                assessment = response_state.get("initial_assessment", {})
                
                # Display response
                print(f"\n🤖 AI Response:")
                print("-" * 60)
                print(ai_response)
                print("-" * 60)
                
                if show_details:
                    print(f"\n📊 Assessment:")
                    print(f"  Context Used: {assessment.get('context_used', 'Unknown')}")
                    print(f"  Confidence: {assessment.get('confidence', 'Unknown')}")
                    print(f"  Response Time: {response_time:.2f}s")
                    print(f"  Next Action: {response_state.get('next_action', 'Unknown')}")
                
                # Store result
                results.append({
                    'question': question,
                    'response': ai_response,
                    'assessment': assessment,
                    'response_time': response_time,
                    'success': True
                })
                
            except Exception as e:
                print(f"❌ Error: {e}")
                results.append({
                    'question': question,
                    'error': str(e),
                    'success': False
                })
                
            if i < len(questions):
                print("\n" + "-"*40)
                
        print(f"\n✅ Testing completed! {sum(1 for r in results if r['success'])}/{len(results)} successful")
        return results
        
    def update_setting(self, category: str, key: str, value: Any):
        """Update a configuration setting temporarily (in memory)"""
        if category == "model":
            if key == "preferred":
                available_models = self.get_available_models()
                if value not in available_models:
                    print(f"❌ Error: Model '{value}' not found. Available models: {', '.join(available_models)}")
                    return
                self.current_settings['agent_config']['models']['preferred'] = value
                print(f"✅ Updated preferred model to: {value}")
            elif key == "fallback":
                if isinstance(value, str):
                    value = [value]
                self.current_settings['agent_config']['models']['fallback'] = value
                print(f"✅ Updated fallback models to: {', '.join(value)}")
                
        elif category == "settings":
            if key in ["temperature", "max_tokens", "timeout", "max_retries"]:
                self.current_settings['agent_config']['settings'][key] = value
                print(f"✅ Updated {key} to: {value}")
            else:
                print(f"❌ Error: Unknown setting '{key}'")
                
        elif category == "behavior":
            if key in ["context_integration", "response_style", "personalization"]:
                self.current_settings['agent_config']['behavior'][key] = value
                print(f"✅ Updated {key} to: {value}")
            else:
                print(f"❌ Error: Unknown behavior setting '{key}'")
                
        elif category == "escalation":
            if key in ["confidence_threshold", "enable_auto_escalation"]:
                self.current_settings['agent_config']['escalation'][key] = value
                print(f"✅ Updated {key} to: {value}")
            else:
                print(f"❌ Error: Unknown escalation setting '{key}'")
        else:
            print(f"❌ Error: Unknown category '{category}'")
            
        # Reinitialize agent with new settings
        self._apply_settings()
        
    def _apply_settings(self):
        """Apply current settings to the agent (reinitialize)"""
        # Save current settings to temporary files
        temp_agent_config = self.config_dir / "agents" / "answer_agent" / "config_temp.yaml"
        
        with open(temp_agent_config, 'w') as f:
            yaml.dump(self.current_settings['agent_config'], f, default_flow_style=False)
            
        # Backup original config
        original_config = self.config_dir / "agents" / "answer_agent" / "config.yaml"
        backup_config = self.config_dir / "agents" / "answer_agent" / "config_backup.yaml"
        
        # Temporarily replace config
        if original_config.exists():
            original_config.rename(backup_config)
        temp_agent_config.rename(original_config)
        
        try:
            # Reinitialize
            self.config_manager = AgentConfigManager(str(self.config_dir))
            self.answer_agent = AnswerAgentNode(self.config_manager, self.context_manager)
            print("🔄 Settings applied successfully!")
        except Exception as e:
            print(f"❌ Error applying settings: {e}")
        finally:
            # Restore original config
            if backup_config.exists():
                if original_config.exists():
                    original_config.unlink()
                backup_config.rename(original_config)
                
    def save_settings(self, permanent: bool = False):
        """Save current settings back to configuration files"""
        if not permanent:
            response = input("⚠️  This will overwrite your configuration files. Continue? (y/N): ")
            if response.lower() != 'y':
                print("❌ Save cancelled.")
                return
                
        try:
            # Save agent config
            agent_config_path = self.config_dir / "agents" / "answer_agent" / "config.yaml"
            with open(agent_config_path, 'w') as f:
                yaml.dump(self.current_settings['agent_config'], f, default_flow_style=False)
                
            # Save models config
            models_config_path = self.config_dir / "shared" / "models.yaml"
            with open(models_config_path, 'w') as f:
                yaml.dump(self.current_settings['models_config'], f, default_flow_style=False)
                
            print("✅ Settings saved successfully!")
            print(f"   - Agent config: {agent_config_path}")
            print(f"   - Models config: {models_config_path}")
            
        except Exception as e:
            print(f"❌ Error saving settings: {e}")
            
    def reset_settings(self):
        """Reset settings to original configuration"""
        print("🔄 Resetting to original configuration...")
        self._load_current_settings()
        self._apply_settings()
        print("✅ Settings reset successfully!")
        
    def show_help(self):
        """Display help information"""
        print("\n" + "="*80)
        print("📚 INTERACTIVE AGENT TESTER - HELP")
        print("="*80)
        print("This tool allows you to test questions and modify agent settings interactively.")
        print("\n🔧 AVAILABLE METHODS:")
        print("  tester.display_current_settings()     - Show current configuration")
        print("  tester.get_available_models()         - List available models")
        print("  tester.test_questions(questions)      - Test a list of questions")
        print("  tester.update_setting(cat, key, val)  - Update a setting temporarily")
        print("  tester.save_settings()                - Save current settings to files")
        print("  tester.reset_settings()               - Reset to original settings")
        print("  tester.show_help()                    - Show this help")
        
        print("\n⚙️  SETTING CATEGORIES:")
        print("  'model' settings:")
        print("    - preferred: str (model name)")
        print("    - fallback: List[str] (list of model names)")
        print("  'settings' options:")
        print("    - temperature: float (0.0-2.0)")
        print("    - max_tokens: int")
        print("    - timeout: int (seconds)")
        print("    - max_retries: int")
        print("  'behavior' options:")
        print("    - context_integration: bool")
        print("    - response_style: str")
        print("    - personalization: bool")
        print("  'escalation' options:")
        print("    - confidence_threshold: float (0.0-1.0)")
        print("    - enable_auto_escalation: bool")
        
        print("\n📝 EXAMPLE USAGE:")
        print("  tester.update_setting('model', 'preferred', 'gpt-4')")
        print("  tester.update_setting('settings', 'temperature', 0.5)")
        print("  tester.update_setting('behavior', 'response_style', 'casual')")
        print("  tester.test_questions(['Hello', 'What is AI?'])")
        print("  tester.save_settings()  # Save when satisfied with results")
        print("="*80)

In [3]:
# Initialize the Interactive Agent Tester
tester = InteractiveAgentTester()
tester.initialize()

# Show help and current settings
tester.show_help()
tester.display_current_settings()

🚀 Initializing Agent Testing System...
✅ 13:37:20.042 [INFO    ] factory         | Attempting to create provider: anthropic_general_budget
✅ Creating LLM provider: anthropic_general_budget → claude-3-5-haiku-20241022 (anthropic)
✅ 13:37:20.045 [INFO    ] claude-3-5-haiku-20241022 | Initializing LLM provider | model_name=claude-3-5-haiku-20241022
✅ 13:37:20.049 [INFO    ] claude-3-5-haiku-20241022 | LLM provider initialized successfully | model_name=claude-3-5-haiku-20241022
✅ 13:37:20.050 [INFO    ] factory         | Successfully created provider: anthropic_general_budget
✅ 13:37:20.051 [INFO    ] answer_agent    | Answer Agent LLM provider initialized | operation=initialize_llm_provider model_name=claude-3-5-haiku-20241022
✅ System initialized successfully!

📚 INTERACTIVE AGENT TESTER - HELP
This tool allows you to test questions and modify agent settings interactively.

🔧 AVAILABLE METHODS:
  tester.display_current_settings()     - Show current configuration
  tester.get_available_mo

In [4]:
# 🧪 INTERACTIVE TESTING SECTION
# Modify this list to test different questions
test_questions = [
    "What is the capital of France?",
    "Explain quantum computing in simple terms.",
    "How do neural networks work?",
    "What are the benefits of renewable energy?",
    "Write a Python function to calculate fibonacci numbers."
]

print("📋 Current test questions:")
for i, q in enumerate(test_questions, 1):
    print(f"  {i}. {q}")

print(f"\n🎯 Available models: {', '.join(tester.get_available_models())}")
print("\n" + "="*80)
print("💡 USAGE EXAMPLES:")
print("="*80)
print("# Test current questions:")
print("results = tester.test_questions(test_questions)")
print()
print("# Modify settings:")
print("tester.update_setting('model', 'preferred', 'mistral-7b')")
print("tester.update_setting('settings', 'temperature', 0.5)")
print("tester.update_setting('behavior', 'response_style', 'casual')")
print()
print("# Save settings when satisfied:")
print("tester.save_settings()")
print()
print("# Reset if needed:")
print("tester.reset_settings()")
print("="*80)

📋 Current test questions:
  1. What is the capital of France?
  2. Explain quantum computing in simple terms.
  3. How do neural networks work?
  4. What are the benefits of renewable energy?
  5. Write a Python function to calculate fibonacci numbers.

🎯 Available models: llama-7b, llama-13b, mistral-7b, codellama-7b, gpt-4, gpt-3.5-turbo, claude-3-5-sonnet-20241022, claude-3-5-haiku-20241022

💡 USAGE EXAMPLES:
# Test current questions:
results = tester.test_questions(test_questions)

# Modify settings:
tester.update_setting('model', 'preferred', 'mistral-7b')
tester.update_setting('settings', 'temperature', 0.5)
tester.update_setting('behavior', 'response_style', 'casual')

# Save settings when satisfied:
tester.save_settings()

# Reset if needed:
tester.reset_settings()


In [5]:
# 🚀 RUN TESTS
# Execute this cell to test your questions with current settings
results = tester.test_questions(test_questions)

# 📊 SUMMARY
print(f"\n📊 TEST SUMMARY:")
print("="*60)
successful_tests = [r for r in results if r['success']]
failed_tests = [r for r in results if not r['success']]

print(f"✅ Successful: {len(successful_tests)}/{len(results)}")
print(f"❌ Failed: {len(failed_tests)}/{len(results)}")

if successful_tests:
    avg_response_time = sum(r['response_time'] for r in successful_tests) / len(successful_tests)
    print(f"⏱️  Average response time: {avg_response_time:.2f}s")

if failed_tests:
    print(f"\n❌ Failed questions:")
    for test in failed_tests:
        print(f"  - {test['question']}: {test.get('error', 'Unknown error')}")

print("="*60)


🧪 TESTING 5 QUESTIONS

--- Question 1/5 ---
❓ What is the capital of France?
⏳ Processing...
✅ 13:37:22.679 [INFO    ] claude-3-5-haiku-20241022 | Model call: claude-3-5-haiku-20241022 - generate_response | model_name=claude-3-5-haiku-20241022 operation=generate_response

🤖 AI Response:
------------------------------------------------------------
The capital of France is Paris. It is located in the north-central part of the country and is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and the Arc de Triomphe. Paris is not only the political and administrative capital of France, but also a global center for art, culture, fashion, and cuisine.
------------------------------------------------------------

📊 Assessment:
  Context Used: False
  Confidence: 0.85
  Response Time: 2.63s
  Next Action: evaluate

----------------------------------------

--- Question 2/5 ---
❓ Explain quantum computing in simple terms.
⏳ Processing...
✅ 13:37:2

In [6]:
# 🔧 INTERACTIVE SETTINGS MODIFICATION
# Use this cell to experiment with different settings

print("🔧 SETTINGS MODIFICATION EXAMPLES")
print("="*60)
print("# Uncomment and modify the lines below to change settings:")
print()

# Model settings
print("# Change model:")
print("# tester.update_setting('model', 'preferred', 'mistral-7b')")
print("# tester.update_setting('model', 'preferred', 'gpt-4')")
print("# tester.update_setting('model', 'fallback', ['mistral-7b', 'llama-7b'])")
print()

# Generation settings  
print("# Adjust generation parameters:")
print("# tester.update_setting('settings', 'temperature', 0.3)  # More focused")
print("# tester.update_setting('settings', 'temperature', 0.9)  # More creative")
print("# tester.update_setting('settings', 'max_tokens', 1000)")
print("# tester.update_setting('settings', 'timeout', 60)")
print()

# Behavior settings
print("# Modify behavior:")
print("# tester.update_setting('behavior', 'response_style', 'casual')")
print("# tester.update_setting('behavior', 'response_style', 'technical')")
print("# tester.update_setting('behavior', 'personalization', False)")
print("# tester.update_setting('behavior', 'context_integration', False)")
print()

# Escalation settings
print("# Adjust escalation:")
print("# tester.update_setting('escalation', 'confidence_threshold', 0.8)")
print("# tester.update_setting('escalation', 'enable_auto_escalation', False)")
print()

print("# View current settings:")
print("# tester.display_current_settings()")
print()
print("# Save settings when satisfied:")
print("# tester.save_settings()")

print("="*60)

# Example: Uncomment these lines to test with a more creative setting
# print("🧪 Testing with higher creativity (temperature=0.9)...")
# tester.update_setting('settings', 'temperature', 0.9)
# creative_results = tester.test_questions(["Write a creative story about AI"])

# Example: Uncomment to test with different model
# print("🧪 Testing with different model...")
# tester.update_setting('model', 'preferred', 'mistral-7b')
# model_results = tester.test_questions(["Explain machine learning"])

🔧 SETTINGS MODIFICATION EXAMPLES
# Uncomment and modify the lines below to change settings:

# Change model:
# tester.update_setting('model', 'preferred', 'mistral-7b')
# tester.update_setting('model', 'preferred', 'gpt-4')
# tester.update_setting('model', 'fallback', ['mistral-7b', 'llama-7b'])

# Adjust generation parameters:
# tester.update_setting('settings', 'temperature', 0.3)  # More focused
# tester.update_setting('settings', 'temperature', 0.9)  # More creative
# tester.update_setting('settings', 'max_tokens', 1000)
# tester.update_setting('settings', 'timeout', 60)

# Modify behavior:
# tester.update_setting('behavior', 'response_style', 'casual')
# tester.update_setting('behavior', 'response_style', 'technical')
# tester.update_setting('behavior', 'personalization', False)
# tester.update_setting('behavior', 'context_integration', False)

# Adjust escalation:
# tester.update_setting('escalation', 'confidence_threshold', 0.8)
# tester.update_setting('escalation', 'enable_auto

In [7]:
# 💾 SAVE SETTINGS TO CONFIG FILES
# Run this cell when you're satisfied with your settings and want to save them permanently

print("💾 SAVE SETTINGS")
print("="*50)
print("Current settings will be saved to:")
print("  - config/agents/answer_agent/config.yaml")
print("  - config/shared/models.yaml")
print()
print("⚠️  Warning: This will overwrite your existing configuration files!")
print()
print("# Uncomment the line below to save settings:")
print("# tester.save_settings()")
print()
print("# Or to save without confirmation:")
print("# tester.save_settings(permanent=True)")
print()
print("# To reset to original settings:")
print("# tester.reset_settings()")
print("="*50)

💾 SAVE SETTINGS
Current settings will be saved to:
  - config/agents/answer_agent/config.yaml
  - config/shared/models.yaml

⚠️  Warning: This will overwrite your existing configuration files!

# Uncomment the line below to save settings:
# tester.save_settings()

# Or to save without confirmation:
# tester.save_settings(permanent=True)

# To reset to original settings:
# tester.reset_settings()
